In [ ]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold


In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

In [ ]:
roundup(1,2)

# generate a file to convert from BD (blood DNA) to FD (fecal DNA) samples
as phoenotypic data tables I get from people in the lab are indexed by FD numbers:

## generate a file with BD numbers and dates 

I queried the database for all BD samples with their userIDs and storage using the following SQL synthas:
select DnaID,UserID,StorageDT from Lab.hostdna 

and saved te results in bloodDNAID_userID_storageDate_111217.csv

In [ ]:
#open this file as dataframe:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/bloodDNAID_userID_storageDate_111217.csv'
BDsWithDates=pd.read_csv(file1)
BDsWithDates.head()

In [ ]:
BDsWithDates[BDsWithDates['DnaID']=='BD78']

now I don't want the userID column as it might be wrong and need to be corrected according to the reg num correction done by Daphna

In [ ]:
BDsWithDates2=BDsWithDates[['DnaID','StorageDT']]
BDsWithDates2.head()

## get corrected reg num for each BD:

In [ ]:
#open the file with corrected userIDs for each BD number based on Daphna's correction for registration number:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples'
CleanCorrectBloodDNASamples=pd.read_pickle(file1)
CleanCorrectBloodDNASamples.head()

In [ ]:
BD_correctReg=CleanCorrectBloodDNASamples[['DnaID','correct registration code','correction status (reg number)']]
BD_correctReg.head()

## Get list of reg num+userIDs:

The following file was generated by the following query to the database: 
select UserID,RegistrationCode from pnp.users 

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/UserIDs_RegNum_171217.csv'
UserIDs_RegNum_171217=pd.read_csv(file1)
UserIDs_RegNum_171217.head()

## merge BD+date with BD+correct reg num:

In [ ]:
BD_date_correctReg=pd.merge(BDsWithDates2,BD_correctReg,how='inner',left_on='DnaID',right_on='DnaID')
print len(BD_date_correctReg)
print len(BDsWithDates2)
print len(BD_correctReg)

In [ ]:
BD_date_correctReg.head()

## merge with user ID:

In [ ]:
BD_date_correctReg_userID=pd.merge(BD_date_correctReg,UserIDs_RegNum_171217,how='left',left_on='correct registration code',
                                   right_on='RegistrationCode')
print len(BD_date_correctReg)
print len(UserIDs_RegNum_171217)
print len(BD_date_correctReg_userID)

In [ ]:
BD_date_correctReg_userID.head()

In [ ]:
BD_date_correctReg_userID[BD_date_correctReg_userID['correction status (reg number)']=='corrected'].head()

In [ ]:
corrected=BD_date_correctReg_userID[BD_date_correctReg_userID['correction status (reg number)']=='corrected']
correctedBDList=list(corrected['DnaID'])

## extract the production/storage year and combine it with the userID to generate an identifier to be compared with the FD sample information

In [ ]:
BD_date_correctReg_userID['Blood_Year']=BD_date_correctReg_userID['StorageDT'].str.split('-').str[0]
BD_date_correctReg_userID['UserID']=BD_date_correctReg_userID['UserID'].astype(str)
BD_date_correctReg_userID['UserID']=BD_date_correctReg_userID['UserID'].str.split('.').str[0]
BD_date_correctReg_userID['UserID_bYear']=BD_date_correctReg_userID['UserID'].astype(str).str.cat(BD_date_correctReg_userID['Blood_Year'],sep='_')
BD_date_correctReg_userID.head()

## save to excel and pickles:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_date_correctReg_userID'
BD_date_correctReg_userID.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_date_correctReg_userIDn.xlsx'
BD_date_correctReg_userID.to_excel(file2)

## generate FD sample file to converge with the BD sample file:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/all_fd_connid.csv'
all_fd_connids=pd.read_csv(file1)
all_fd_connids.head()


In [ ]:
all_fd_connid2=all_fd_connids[['DnaID','UserID','StorageDT']]
all_fd_connid2.head()

In [ ]:
all_fd_connid2['f_Year']=all_fd_connid2['StorageDT'].str.split('-').str[0]
all_fd_connid2['UserID']=all_fd_connid2['UserID'].astype(str)
all_fd_connid2['UserID']=all_fd_connid2['UserID'].str.split('.').str[0]
all_fd_connid2['UserID_fYear']=all_fd_connid2['UserID'].astype(str).str.cat(all_fd_connid2['f_Year'],sep='_')
all_fd_connid2.head()

combine the relevant columns from the BD and FD tables according to the userID_year combination:

In [ ]:
BDsWithDates2=BD_date_correctReg_userID[['DnaID','UserID','UserID_bYear']]
all_fd_connid3=all_fd_connid2[['DnaID','UserID','UserID_fYear']]

In [ ]:
BD_FD=pd.merge(BDsWithDates2,all_fd_connid3,how='left',left_on=['UserID','UserID_bYear'],
               right_on=['UserID','UserID_fYear'])

In [ ]:
BD_FD.head()

In [ ]:
print len(BD_FD)

In [ ]:
#rename columns:
BD_FD=BD_FD.rename(columns={'DnaID_x':'BD','DnaID_y':'FD','UserID_bYear':'UserID_Year_BDsample',
                           'UserID_fYear':'UserID_Year_FDsample','UserID_x':'UserID_BDsample',
                            'UserID_y':'UserID_FDsample'})
BD_FD.head()

## go over all BD samples that don't have matched FD sample and try to find FD sample by userID only
add comment column, and mark that the years are not the same between the BD and FD sample

In [ ]:
BD_FD[BD_FD['BD']=='BD344']

In [ ]:
BD_FD['Comment']=''
for n in BD_FD[BD_FD['FD'].isnull()].index:
        userID=BD_FD.loc[n,'UserID']
        relevantDF=all_fd_connid3[all_fd_connid3['UserID']==userID]
        DFlength=len(relevantDF)
        FDlist=list(relevantDF['DnaID'])
        UserFlist=list(relevantDF['UserID'])
        FDyearList=list(relevantDF['UserID_fYear'])
        BDlist=BD_FD.loc[n,'BD']
        UserID_Year_BDsample_list=BD_FD.loc[n,'UserID_Year_BDsample']
        newDF=pd.DataFrame({'BD':BDlist,'UserID':userID,'UserID_Year_BDsample':UserID_Year_BDsample_list,
                          'FD':FDlist,'UserID_Year_FDsample': FDyearList,'Comment':'Not the same year!!'})
        
        print newDF.columns.values
        print BD_FD.columns.values
        BD_FD=pd.concat([BD_FD,newDF])
        

    

In [ ]:
final_BD_FD=BD_FD[BD_FD['FD'].notnull()]
final_BD_FD=final_BD_FD.reset_index()
final_BD_FD

In [ ]:
print len(final_BD_FD)

## save into pickle and excel files

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_FD_sample_conversion'
final_BD_FD.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_FD_sample_conversion.xlsx'
final_BD_FD.to_excel(file2)

## sanity check for samples that their userIDs were corrected:

In [ ]:
correctBD_FD=final_BD_FD.set_index('BD')
correctBD_FD=correctBD_FD.loc[correctedBDList,:]
correctBD_FD.head()

In [ ]:
for n in final_BD_FD.index:
    if str(final_BD_FD.loc[n,'UserID']) not in str(final_BD_FD.loc[n,'UserID_Year_BDsample']):
        print final_BD_FD.loc[n,:]



In [ ]:
for n in final_BD_FD.index:
    if str(final_BD_FD.loc[n,'UserID']) not in str(final_BD_FD.loc[n,'UserID_Year_FDsample']):
        print final_BD_FD.loc[n,:]



# Index the phenotypic data I got from Liron by BD numbers

## download files

In [ ]:
# the table from Liron;
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/full_x_nov19.csv'
full_x_nov19=pd.read_csv(file1)
full_x_nov19.head()

In [ ]:
full_x_nov19.columns.values

In [ ]:
for column in full_x_nov19.columns.values:
    if full_x_nov19[column].dtype!='float64':
        print column, full_x_nov19[column].dtype

## merge BD numbers into the table:

In [ ]:
BD_FD_to_use=final_BD_FD[['BD','FD','Comment','UserID','UserID_Year_BDsample']]

In [ ]:
fullXwithBDs=pd.merge(full_x_nov19,BD_FD_to_use,how='left',left_on='FD',right_on='FD')
print len(fullXwithBDs)
print len(full_x_nov19)
print len(BD_FD_to_use)

In [ ]:
fullXwithBDs.head()

## group data by BD numbers

check if there are categorial variables that can't be averaged: (the answer is no)

In [ ]:
for column in fullXwithBDs.columns.values:
    if fullXwithBDs[column].dtype!='float64':
        print column, fullXwithBDs[column].dtype

Generate a list of the binary variables:

in case of several entries to the same BD sample - average the other parameters

In [ ]:
fullXgroupbyBD=fullXwithBDs.groupby('BD').mean()
fullXgroupbyBD.head()

generate FD lists per BD and integrate them to the file:

In [ ]:

FDlistperBD=pd.DataFrame(fullXwithBDs.groupby('BD')['FD'].apply(list))
FDlistperBD.head()


In [ ]:
fullXgroupbyBD=pd.merge(fullXgroupbyBD,FDlistperBD,how='left',right_index=True, left_index=True)
fullXgroupbyBD

check the grouping of the binary variables:


In [ ]:
binaryVariableList=[]
for column in fullXwithBDs.columns.values:
    values=fullXwithBDs[column].unique()
    if len(values)<4:
        print column, values
        binaryVariableList.append(column)

binaryVariableList.remove('Comment')

In [ ]:

print binaryVariableList

In [ ]:
fullXgroupbyBD[binaryVariableList].head()

In [ ]:
binary=fullXgroupbyBD[binaryVariableList]
for column in binary.columns.values:
    print column, binary[column].unique()
        

***most of the binary variables kept their binary format after the grouping, except for the variables: Regular defecation,Regular period,IsGenotek***

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/binaryVariableList.txt'
with open(file1, "wb") as fp:   #Pickling
    pickle.dump(binaryVariableList, fp)



## clean the binary variables:

In [ ]:
fullXgroupbyBD.head()

In [ ]:
fullXgroupbyBD['IsGenotek'].value_counts()

In [ ]:

for column in binary.columns.values:
    condition1=fullXgroupbyBD[column]==0
    condition2=fullXgroupbyBD[column]==1
    fullXgroupbyBD[column]=np.where(condition1,0,(np.where(condition2,1,np.nan)))
    

In [ ]:
fullXgroupbyBD['IsGenotek'].value_counts()

## read files

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXgroupbyBD'
fullXgroupbyBD=pd.read_pickle(file1)

## save files:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXgroupbyBD'
fullXgroupbyBD.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXgroupbyBD.xlsx'
fullXgroupbyBD.to_excel(file2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXwithBDs'
fullXwithBDs.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXwithBDs.xlsx'
fullXwithBDs.to_excel(file2)

## sanity check for the feature files

In [ ]:
fullXwithBDs.columns.values

UserIDs with many FD samples - top 5:

In [ ]:
fullXwithBDs.groupby('UserID').count()['FD'].sort_values().tail()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD'
SampleSpeciesDFgroupedByBD.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD.xlsx'
SampleSpeciesDFgroupedByBD.to_excel(file2)

BDs with many FD samples - top 5:

In [ ]:
fullXwithBDs.groupby('BD').count()['FD'].sort_values().tail()

FD samples  with many BD samples - top 5:

In [ ]:
fullXwithBDs.groupby('FD').count()['BD'].sort_values().tail()

In [ ]:
fullXwithBDs.groupby('FD').count()['UserID'].sort_values().tail()

### get lists of BDs per userIDs for users with more than one BD:

In [ ]:
BDlistperUserID=pd.DataFrame(fullXwithBDs.groupby('UserID')['BD'].apply(list))
BDlistperUserID_DF=pd.DataFrame()
for n,user in enumerate(BDlistperUserID.index):
    uniqueBDsPerUserID=list(set(BDlistperUserID.loc[user,'BD']))
    if len(uniqueBDsPerUserID)>1:
#         print str(uniqueFDsPerUserID)
        BDlistperUserID_DF.loc[n,'UserID']=user
        BDlistperUserID_DF.loc[n,'BDs']=str(uniqueBDsPerUserID)
        BDlistperUserID_DF.loc[n,'n_BDs']=len(uniqueBDsPerUserID)
BDlistperUserID_DF=BDlistperUserID_DF.sort_values(by='n_BDs', ascending=False)
BDlistperUserID_DF

### get lists of FDs per userIDs for users with more than one FD:

In [ ]:
FDlistperUserID=pd.DataFrame(fullXwithBDs.groupby('UserID')['FD'].apply(list))
FDlistperUserID_DF=pd.DataFrame()
for n,user in enumerate(FDlistperUserID.index):
    uniqueFDsPerUserID=list(set(FDlistperUserID.loc[user,'FD']))
    if len(uniqueFDsPerUserID)>1:
#         print str(uniqueFDsPerUserID)
        FDlistperUserID_DF.loc[n,'UserID']=user
        FDlistperUserID_DF.loc[n,'FDs']=str(uniqueFDsPerUserID)
        FDlistperUserID_DF.loc[n,'n_FDs']=len(uniqueFDsPerUserID)
FDlistperUserID_DF=FDlistperUserID_DF.sort_values(by='n_FDs', ascending=False)
FDlistperUserID_DF

### get lists of FDs per BD for BDs with more than one FD:

In [ ]:
FDlistperBD=pd.DataFrame(fullXwithBDs.groupby('BD')['FD'].apply(list))
FDlistperBD_DF=pd.DataFrame()
for n,BD in enumerate(FDlistperBD.index):
    uniqueFDsPerBD=list(set(FDlistperBD.loc[BD,'FD']))
    if len(uniqueFDsPerBD)>1:
#         print str(uniqueFDsPerUserID)
        FDlistperBD_DF.loc[n,'BD']=BD
        FDlistperBD_DF.loc[n,'FDs']=str(uniqueFDsPerBD)
        FDlistperBD_DF.loc[n,'n_FDs']=len(uniqueFDsPerBD)
FDlistperBD_DF=FDlistperBD_DF.sort_values(by='n_FDs', ascending=False)
FDlistperBD_DF

# process phenotypes from Izhak

In [ ]:
# the table from Izhak;
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/PhenotypesIzhak.dat'
PhenotypesIzhak=pd.read_pickle(file1)
PhenotypesIzhak.head()

In [ ]:
PhenotypesIzhak.columns.values

this phenotypes are not so interesting at this stage!!

# get some cohort phenotypes:

## generate df only for samples we already sequenced:

In [ ]:
data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print filenames[:5]

### generate list of PNP434 samples without '_' and 'b' in the sample name:

In [ ]:
PNP434samples=[]
for sample in filenames:
    if '_' in sample:
        NewName=sample.split('_')[0]
    else:
        NewName=sample
    if 'b' in NewName:
        print NewName
        NewName=NewName.split('b')[0]
        print NewName 
    PNP434samples.append(NewName)
# PNP434samples

In [ ]:
print len(PNP434samples)

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/PNP434samples'
with open(file3,'wb') as fp:
    pickle.dump(PNP434samples,fp)

In [ ]:
fullXgroupbyBD_only434=fullXgroupbyBD.loc[PNP434samples,:]

In [ ]:
fullXgroupbyBD_only434.head()

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/fullXgroupbyBD_only434'
fullXgroupbyBD_only434.to_pickle(file1)

## calculate descriptive statistice for all cohort and for the 434 cohort:

In [ ]:
def roundup2(a, digits=0):
    n = 10**-digits
    return round(math.ceil(a / n) * n, digits)


numericalFeatures=['Age','Hemoglobin','BMI','CRP (WIDE RANGE)']
binaryFeatures=['Gender','Ever smoked']

fig1=plt.figure(figsize=(8,18))
fig1.suptitle('Main Phenotype Distributions', fontsize=22)

for n,feature in enumerate(numericalFeatures):
    print n,feature
    dataAll=fullXgroupbyBD[feature]
    dataAll=list(dataAll[~np.isnan(dataAll)])
    weightsAll=np.ones_like(dataAll)/len(dataAll)
    meanAll=round(np.mean(dataAll),2)
    stdAll=round(np.std(dataAll),2)
    
    data434=fullXgroupbyBD_only434[feature]
    data434=list(data434[~np.isnan(data434)])
    weights434=np.ones_like(data434)/len(data434)
    mean434=round(np.mean(data434),2)
    std434=round(np.std(data434),2)
    
    Xmin=min(dataAll)
    Xmax=max(dataAll)
    freqs,bins=np.histogram(dataAll,range=(Xmin,Xmax),bins=10,weights=weightsAll)
    Ymax=roundup2(np.max(freqs),1)
     
        
    ax= fig1.add_subplot(6,2,2*n+1)
    
    plot=ax.hist(dataAll,range=(Xmin,Xmax),bins=10,weights=weightsAll) 
    ax.set_xlabel(feature)
    ax.set_ylim(0,Ymax)
    ax.annotate("mean=%s+-%s" %(meanAll,stdAll),  xy=(0.02, 0.96), xycoords='axes fraction', fontsize=12,
    horizontalalignment='left', verticalalignment='top', fontweight='bold')

    ax= fig1.add_subplot(6,2,2*n+2)
    plot=ax.hist(data434,range=(Xmin,Xmax),bins=10,weights=weights434)
    ax.set_xlabel(feature)
    ax.set_ylim(0,Ymax)
    ax.annotate("mean=%s+-%s" %(mean434,std434),  xy=(0.02, 0.96), xycoords='axes fraction', fontsize=12,
    horizontalalignment='left', verticalalignment='top', fontweight='bold')

for n,Bfeature in enumerate(binaryFeatures):
    print n,Bfeature
    if Bfeature=='Gender':
        ticklabels=['Male','Female']
    elif Bfeature=='Ever smoked':
        ticklabels=['No','Yes']
    
    ax= fig1.add_subplot(6,2,2*n+9)   
    a=fullXgroupbyBD[Bfeature].value_counts(normalize=True)
    Ymax=roundup2(a.max(), 1)
    ax.bar([0,1],a,align='center',width=0.6,tick_label=ticklabels)
    ax.set_xlabel(Bfeature)
    ax.set_ylim(0,Ymax)
    
    ax= fig1.add_subplot(6,2,2*n+10)   
    a=fullXgroupbyBD_only434[Bfeature].value_counts(normalize=True)
    ax.bar([0,1],a,align='center',width=0.6,tick_label=ticklabels)
    ax.set_xlim(-0.5,1.5)
    ax.set_xlabel(Bfeature)
    ax.set_ylim(0,Ymax)
    
    




fig1.text(0,0.5,"Frequency",  ha = 'left',fontsize=18, rotation=90)
fig1.text(0.25,0.94,"All Samples",  ha = 'center',fontsize=18)
fig1.text(0.75,0.94,"434 cohort",  ha = 'center',fontsize=18)


fig1.subplots_adjust(left=0.09, right=0.98, top=0.92, bottom=0.02, wspace=0.22,hspace=0.30)



filename='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/MainPhenotypeDistribution_%s' %cdate
fig1.savefig(filename, bbox_inches='tight', dpi = 200)    
    

plt.show()




In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Distance Matrices/PNP434_Age_EucDistMat'
PNP434_Age_EucDistMat.to_pickle(file1)

# generate microbiome species phenotype table:

## load the table from Noam:

In [ ]:
MAP = '/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/MPASpid.dat'

m = pd.read_pickle(MAP)
SampleSpeciesDF=m.loc['s'] 
SampleSpeciesDF=SampleSpeciesDF.T
SampleSpeciesDF.head()
# only species
# species over samples

In [ ]:
print len(SampleSpeciesDF)

## check indices:

In [ ]:
for n in SampleSpeciesDF.index:
    if 'FD999' in n:
        print n

the connection IDs seems to be wrong.
generate column of only FD with split, and check whether there are repeats


## merge file with BD_FD file based on FD numbers ONLY!

In [ ]:
SampleSpeciesDF=SampleSpeciesDF.reset_index()

In [ ]:
SampleSpeciesDF.head()

In [ ]:
print len(SampleSpeciesDF)

In [ ]:
SampleSpeciesDF['FD']=SampleSpeciesDF['index'].str.split('_').str[0]

In [ ]:
SampleSpeciesDF.head()

In [ ]:
#there are no repeating FDs!
SampleSpeciesDF['FD'].value_counts().tail()

In [ ]:
'FD235' in list(SampleSpeciesDF['FD'])

In [ ]:
#load BD_FD sample 

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_FD_sample_conversion'
final_BD_FD=pd.read_pickle(file1)

In [ ]:
BD_FD_to_use=final_BD_FD[['BD','FD','Comment','UserID','UserID_Year_BDsample']]

In [ ]:
SampleSpeciesDFwithBDs=pd.merge(SampleSpeciesDF,BD_FD_to_use,how='left',left_on='FD',right_on='FD')
print len(SampleSpeciesDFwithBDs)
print len(SampleSpeciesDF)
print len(BD_FD_to_use)

In [ ]:
SampleSpeciesDFwithBDs.head()

In [ ]:
SampleSpeciesDFwithBDs=SampleSpeciesDFwithBDs.fillna(0)
SampleSpeciesDFwithBDs.head()

In [ ]:
SampleSpeciesDFwithBDs=SampleSpeciesDFwithBDs[SampleSpeciesDFwithBDs['BD']!=0]
SampleSpeciesDFwithBDs.head()

In [ ]:
print len(SampleSpeciesDFwithBDs)

In [ ]:
'FD238_32' in list(SampleSpeciesDFwithBDs['index'])

In [ ]:
'FD235' in list(SampleSpeciesDFwithBDs['FD'])

In [ ]:
'FD959' in list(SampleSpeciesDFwithBDs['FD'])

In [ ]:
SampleSpeciesDFwithBDs[SampleSpeciesDFwithBDs['FD']=='FD235'

In [ ]:
#save file: 

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFwithBDs'
SampleSpeciesDFwithBDs.to_pickle(file1)

check if there are categorial variables that can't be averaged: (the answer is no)

In [ ]:
for column in SampleSpeciesDFwithBDs.columns.values:
    if SampleSpeciesDFwithBDs[column].dtype!='float64':
        print column, SampleSpeciesDFwithBDs[column].dtype

in case of several entries to the same BD sample - average the other parameters

In [ ]:
SampleSpeciesDFgroupedByBD=SampleSpeciesDFwithBDs.groupby('BD').mean()
SampleSpeciesDFgroupedByBD.head()

In [ ]:
print len(SampleSpeciesDFgroupedByBD)

generate FD lists per BD and integrate them to the file:

In [ ]:

FDlistperBD2=pd.DataFrame(SampleSpeciesDFwithBDs.groupby('BD')['FD'].apply(list))
FDlistperBD2.head()


In [ ]:
SampleSpeciesDFgroupedByBD=pd.merge(SampleSpeciesDFgroupedByBD,FDlistperBD2,how='left',right_index=True, left_index=True)
SampleSpeciesDFgroupedByBD.head()

check the grouping of the binary variables:


In [ ]:
binaryVariableList2=[]
for column in SampleSpeciesDFwithBDs.columns.values:
    values=SampleSpeciesDFwithBDs[column].unique()
    if len(values)<4:
#         print column, values
        binaryVariableList2.append(column)

binaryVariableList2.remove('Comment')

In [ ]:

# print binaryVariableList2

## prepare sampleSpecies DF with only the 434 samples:

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/PNP434samples'
with open(file3,'rb') as fp:
    PNP434samples=pickle.load(fp)

In [ ]:
SampleSpeciesDFgroupedByBD_only434=SampleSpeciesDFgroupedByBD.loc[PNP434samples,:]
print len(SampleSpeciesDFgroupedByBD_only434)
SampleSpeciesDFgroupedByBD_only434=SampleSpeciesDFgroupedByBD_only434.dropna(axis=(0,1),how='all')
print len(SampleSpeciesDFgroupedByBD_only434)

In [ ]:
SampleSpeciesDFgroupedByBD_only434.head()

## transform SampleSpeciesDFgroupedByBD_only434 to log2 scale and save:

In [ ]:
for column in SampleSpeciesDFgroupedByBD_only434.columns.values:

    SampleSpeciesDFgroupedByBD_only434[column]=pd.to_numeric(SampleSpeciesDFgroupedByBD_only434[column],errors='coerce')



In [ ]:
SampleSpeciesDFgroupedByBD_only434_log2scale = SampleSpeciesDFgroupedByBD_only434.applymap(np.log2)

In [ ]:
SampleSpeciesDFgroupedByBD_only434_log2scale.head()

In [ ]:
inf=SampleSpeciesDFgroupedByBD_only434_log2scale.loc['BD438','Abiotrophia_defectiva']

In [ ]:
SampleSpeciesDFgroupedByBD_only434_log2scale=SampleSpeciesDFgroupedByBD_only434_log2scale.replace(inf,-1000)
SampleSpeciesDFgroupedByBD_only434_log2scale.head()

## save files:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFwithBDs'
SampleSpeciesDFwithBDs.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFwithBDs.xlsx'
SampleSpeciesDFwithBDs.to_excel(file2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD'
SampleSpeciesDFgroupedByBD.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD.xlsx'
SampleSpeciesDFgroupedByBD.to_excel(file2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434'
SampleSpeciesDFgroupedByBD_only434.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434.xlsx'
SampleSpeciesDFgroupedByBD_only434.to_excel(file2)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434_log2scale'
SampleSpeciesDFgroupedByBD_only434_log2scale.to_pickle(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/SampleSpeciesDFgroupedByBD_only434_log2scale.xlsx'
SampleSpeciesDFgroupedByBD_only434_log2scale.to_excel(file2)

In [ ]:
SampleSpeciesDFgroupedByBD_only434

# check phenotypic data from the database to validate the fullX file

for several phenotypes, there is data for only small number of samples. for example, total protein data is available for only 18 samples out of the 434 that I work with.
to check if this correlates with the data in the database, I extracted directly from the DB using SQL the pnp.questionnaire and pnp.questions tables and looked for how many subjects we have the relevant data. 

check for how many users we have data for question 1089 - total protein:

In [ ]:
proteintotal=questWitjLookup_bloodTest[questWitjLookup_bloodTest['QuestionID']==1089]
print len(proteintotal)

In [ ]:
print len(proteintotal['UserID'].unique())

In [ ]:
print len(questWitjLookup_bloodTest['UserID'].unique())

In [ ]:
# there is total protein data for 1.5% of the unique UserIDs
float(25)/1642

In [ ]:
# In the PNP434 cohort, I found total protein data for 4.1% of the unique UserIDs

float(18)/434

check for how many users we have data for question 1013 - CRP (WIDE RANGE):

In [ ]:
CRPwide=questWitjLookup_bloodTest[questWitjLookup_bloodTest['QuestionID']==1013]
print len(CRPwide)

In [ ]:
print len(CRPwide['UserID'].unique())

In [ ]:
# there is CRPwide data for 36% of the unique UserIDs
float(589)/1642

In [ ]:
# In the PNP434 cohort, I found total protein data for 53% of the unique UserIDs

float(232)/434

##### check for how many users we have data for question 1085 - iron:

In [ ]:
iron=questWitjLookup_bloodTest[questWitjLookup_bloodTest['QuestionID']==1085]
print len(iron)

In [ ]:
print len(iron['UserID'].unique())

In [ ]:
# there is iron data for 36% of the unique UserIDs
float(20)/1642

In [ ]:
# In the PNP434 cohort, I found iron data for 53% of the unique UserIDs

float(18)/434

# generate a new phenotype file by merging files that I extracted from the DB:

### pnp.questionaire table extracted on 3/1/18

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/pnp_questionaire table from the database.csv'
quest=pd.read_csv(file1)
print len(quest)
quest.head()

### pnp.questions table extracted on 3/1/18

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/pnp_questions_lookupTable_fromDB.csv'
lookup=pd.read_csv(file2)
print len(lookup)
lookup.head()

### merging and processing the table:
dropping unncessary rows
extract the yesr and generate a UserID_year identifier

In [ ]:
questWithLookup=pd.merge(quest,lookup,how='left',left_on='QuestionID',right_on='QuestionID')
print len(questWithLookup)
questWithLookup.head()

In [ ]:
questWithLookup=questWithLookup[['UserID','QuestionID','Question','Answer','Timestamp']]
questWithLookup.head()

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/questWithLookup.csv'
questWithLookup.to_csv(file3)

In [ ]:
interestingQuestions=range(1005,1039)+range(1083,1092)+range(1680,1689)+[1045]


In [ ]:
interestingQuestionsDF=pd.DataFrame(interestingQuestions)
interestingQuestionsDF.head()

In [ ]:
questWithLookup_Interesting=pd.merge(interestingQuestionsDF,questWithLookup,how='left',left_on=0,right_on='QuestionID')
print len(questWithLookup_Interesting)
questWithLookup_Interesting.head()

In [ ]:
for n in questWithLookup_Interesting.index:
    m=re.search('(?<=20)\w+', questWithLookup_Interesting.loc[n,'Timestamp'])
    if m is not None:
        questWithLookup_Interesting.loc[n,'year']='20'+m.group(0)


In [ ]:
questWithLookup_Interesting.tail()

In [ ]:
questWithLookup_Interesting=questWithLookup_Interesting.drop(0,axis=1)

no rows with no userID or quedtionID

In [ ]:
questWithLookup_Interesting[questWithLookup_Interesting['QuestionID'].isnull()]

In [ ]:
file4='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/questWithLookup_Interesting.csv'
questWithLookup_Interesting.to_csv(file4)

In [ ]:
questWithLookup_Interesting['year'].value_counts()

In [ ]:
questWithLookup_Interesting['UserID_Year']=questWithLookup_Interesting['UserID'].astype(str).str.cat(questWithLookup_Interesting['year'],sep='_')

In [ ]:
questWithLookup_Interesting.head()

### merge with BD table:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD_date_correctReg_userID'
BD_date_correctReg_userID=pd.read_pickle(file1)
BD_date_correctReg_userID.head()

In [ ]:
BD_date_correctReg_userID[BD_date_correctReg_userID['DnaID']=='BD345']

In [ ]:
BD_to_use=BD_date_correctReg_userID[['DnaID','UserID','UserID_bYear']]

In [ ]:
BD_to_use['UserID']=BD_to_use['UserID'].replace('nan',np.nan)
BD_to_use=BD_to_use[BD_to_use['UserID'].notnull()]
BD_to_use['UserID']=BD_to_use['UserID'].astype(int)

In [ ]:
BD_to_use[BD_to_use['UserID']==82]

In [ ]:
BD_to_use[BD_to_use['DnaID']=='BD345']

In [ ]:
newPhenotypeTableWithBDs=pd.merge(questWithLookup_Interesting,BD_to_use,how='left',left_on='UserID_Year',right_on='UserID_bYear')

*** very important!!! now we need to delete BD nummbers from row in which the UserID_year is nan ***

In [ ]:
newPhenotypeTableWithBDs['DnaID']=np.where(newPhenotypeTableWithBDs['UserID_Year'].isnull(),np.nan,
                                          newPhenotypeTableWithBDs['DnaID'])

In [ ]:
print len(newPhenotypeTableWithBDs)
newPhenotypeTableWithBDs.head()

In [ ]:
print len(newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID']=='BD345'])
newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID']=='BD345']

### go over all rows that don't have matched BD sample and try to find BD sample by userID only
add comment column, and mark that the years are not the same between the BD and FD sample

In [ ]:
problem=newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID'].isnull()]
problem=problem.rename(columns={'UserID_x':'UserID'})
problem=problem[['UserID','QuestionID','Question','Answer','year','UserID_Year','UserID_bYear']]
problem.head()

In [ ]:
mergedByUserOnly=pd.merge(problem,BD_to_use,how='left',left_on='UserID',right_on='UserID')
mergedByUserOnly.head()

In [ ]:
print len(mergedByUserOnly)
mergedByUserOnly=mergedByUserOnly[mergedByUserOnly['DnaID'].notnull()]
print len(mergedByUserOnly)

In [ ]:
mergedByUserOnly['comment']='Not the same year'
newPhenotypeTableWithBDs['comment']=''

In [ ]:
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs.rename(columns={'UserID_x':'UserID'})
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs[['UserID','DnaID','QuestionID','Question','Answer','Timestamp','year','UserID_Year','UserID_bYear','comment']]

In [ ]:
print len(newPhenotypeTableWithBDs)

In [ ]:
newPhenotypeTableWithBDs.head()

In [ ]:
print len(newPhenotypeTableWithBDs)
newPhenotypeTableWithBDs=pd.concat([newPhenotypeTableWithBDs,mergedByUserOnly])
print len(newPhenotypeTableWithBDs)
newPhenotypeTableWithBDs.head()

In [ ]:
print len(newPhenotypeTableWithBDs)
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID'].notnull()]
print len(newPhenotypeTableWithBDs)

In [ ]:
newPhenotypeTableWithBDs.head()

In [ ]:
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs[['DnaID','UserID','UserID_Year','UserID_bYear','year','QuestionID',
                                                  'Question','Answer']]
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs.reset_index()
newPhenotypeTableWithBDs=newPhenotypeTableWithBDs.drop('index',axis=1)
print len(newPhenotypeTableWithBDs)
newPhenotypeTableWithBDs.head()

### sanity check for this file:

In [ ]:
print len(newPhenotypeTableWithBDs['DnaID'].unique())
print len(newPhenotypeTableWithBDs['UserID'].unique())

In [ ]:
BD_userID=newPhenotypeTableWithBDs[['DnaID','UserID']]

In [ ]:
BD_userID=BD_userID.drop_duplicates()

In [ ]:
print len(newPhenotypeTableWithBDs['DnaID'].unique())
print len(newPhenotypeTableWithBDs['UserID'].unique())

In [ ]:
groups=BD_userID.groupby('UserID')

In [ ]:
for name,group in groups:
    print name,group

In [ ]:
newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['UserID']==26]

In [ ]:
questionList=newPhenotypeTableWithBDs['Question'].unique()
print len(questionList)

In [ ]:
print len(newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID']=='BD345'])
newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['DnaID']=='BD345']

In [ ]:
file3='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTableWithBDs.csv'
newPhenotypeTableWithBDs.to_csv(file3)

### AVERAGE ANSWERS OVER BDs

In [ ]:
#check which questions have very few answers and should be removed from the file
for question in questionList:
    df=newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['Question']==question]
    if len(df['Answer'].unique())<5:
        print question
        print df['Answer'].value_counts()

    

### organize the file so each question has its own column
do not include questions with less than 5 unique answers
average answers for the same question in the same BD sample

In [ ]:
newPhenotypeTable_groupedByBD=pd.DataFrame(newPhenotypeTableWithBDs['DnaID'].unique())
for n,question in enumerate(questionList):
    df=newPhenotypeTableWithBDs[newPhenotypeTableWithBDs['Question']==question]
    if len(df['Answer'].unique())>4:
        print question
        df=df[['DnaID','Answer']]
        df['Answer']=pd.to_numeric(df['Answer'],errors='coerce')
        df=df.rename(columns={'Answer':question})
        grouped=df.groupby('DnaID').mean()
        
        if n==0:
            newPhenotypeTable_groupedByBD=grouped
        else:
            newPhenotypeTable_groupedByBD=pd.merge(newPhenotypeTable_groupedByBD,grouped,how='left',
                                                   left_index=True,right_index=True)
    else:
        print '%s have only %s values and was not included' %(question, len(df['Answer'].unique()))    

In [ ]:
newPhenotypeTable_groupedByBD.head()

In [ ]:
describe=newPhenotypeTable_groupedByBD.describe()
describe

In [ ]:
for column in describe.columns.values:
    if describe.loc['count',column]<50:
        print column
        print describe.loc['count',column]
    if 'Neutrophils' in column:
        print column
        print describe.loc['count',column]
    

In [ ]:
newPhenotypeTable_groupedByBD=newPhenotypeTable_groupedByBD.drop('Urine Erythrocytes',axis=1)
newPhenotypeTable_groupedByBD=newPhenotypeTable_groupedByBD.drop('Neutrophils No.',axis=1)

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTable_groupedByBD.xlsx'
newPhenotypeTable_groupedByBD.to_excel(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTable_groupedByBD'
newPhenotypeTable_groupedByBD.to_pickle(file2)

In [ ]:
newPhenotypeTable_groupedByBD_only434=newPhenotypeTable_groupedByBD.loc[PNP434samples,:]

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTable_groupedByBD_only434.xlsx'
newPhenotypeTable_groupedByBD_only434.to_excel(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/newPhenotypeTable_groupedByBD_only434'
newPhenotypeTable_groupedByBD_only434.to_pickle(file2)

# compare the new phenotype file to Liron's file
look for columns that doesn't exist in Liron's file
look for column that exist but I have more data

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/fullXgroupbyBD'
fullXgroupbyBD=pd.read_pickle(file1)

In [ ]:
oldColumns=fullXgroupbyBD.columns.values
newColumns=newPhenotypeTable_groupedByBD.columns.values

In [ ]:
allcolumns=list(np.unique(list(oldColumns)+list(newColumns)))
allcolumns.remove('FD')

In [ ]:
describeNew=newPhenotypeTable_groupedByBD.describe()
describeNew

In [ ]:
describeOld=fullXgroupbyBD.describe()
describeNew=newPhenotypeTable_groupedByBD.describe()


columnsList=allcolumns
nSamplesOldList=[]
nSamplesNewList=[]

for column in allcolumns:
    print column
    
    
    if column in oldColumns:
        print describeOld.loc['count',column]
        if column!='FD':
            nSamplesOld=describeOld.loc['count',column]
            nSamplesOldList.append(nSamplesOld)
    else:
        nSamplesOldList.append(np.nan)
        
    if column in newColumns:
        print describeNew.loc['count',column]
        nSamplesNew=describeNew.loc['count',column]
        nSamplesNewList.append(nSamplesNew)
    else:
        nSamplesNewList.append(np.nan)

In [ ]:
columnsDF=pd.DataFrame({'Column':columnsList,'nSamplesOld':nSamplesOldList,'nSamplesNew':nSamplesNewList})
columnsDF                      

In [ ]:
columnsDF[columnsDF['nSamplesOld']>columnsDF['nSamplesNew']]

In [ ]:
TotalNotInProd=[column for column in TotalColumns if column not in ProdColumns ]

In [ ]:
columnsToAnalyzeFromNewFile=list(columnsDF[columnsDF['nSamplesNew'].notnull()]['Column'])

In [ ]:
columnsToAnalyzeFromNewFile

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/PhenotypicData/columnsToAnalyzeFromNewFile'
with open(file1,'wb') as fp:
    pickle.dump(columnsToAnalyzeFromNewFile,fp)